In [1]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

%useLatestDescriptors
%use lets-plot

class Main

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data

In [2]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val vulnCollection = db.getCollection<BsonDocument>("mergedVuln")

In [3]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [4]:
import org.bson.BsonNull
import org.bson.BsonString
import org.jetbrains.kotlinx.dataframe.math.mean
import org.jetbrains.kotlinx.dataframe.math.median

val vulnGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        gav to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()

println(vulnGavToClasses.count())
println(vulnGavToClasses.map { it.value.size }.minOrNull())
println(vulnGavToClasses.map { it.value.size }.maxOrNull())
println(vulnGavToClasses.map { it.value.size }.average())
println(vulnGavToClasses.map { it.value.size }.median())

3621
1
51
3.3526650096658384
2


In [5]:
val vulnCveGavToClasses = vulnCollection.find().useCursor { blk ->
    blk.map {
        val gav = it["vuln_gav"]!!.asString().value
        val cve = it["cve_ref"]!!.asString().value.let { c -> if (c.isBlank()) it["snyk_url"]!!.asString().value else c }
        val vulnClasses = it["vuln_classes"]!!.asArray().map { it.asString().value }.toSet()

        if (vulnClasses.count() == 0) throw Exception("no vuln class (record should have been ommitted previously)")

        (cve to gav) to vulnClasses
    }
}.groupBy { it.first }.map { g -> g.key to g.value.flatMap { it -> it.second }.toSet() }.toMap()
    
println(vulnCveGavToClasses.count())
println(vulnCveGavToClasses.map { it.value.size }.minOrNull())
println(vulnCveGavToClasses.map { it.value.size }.maxOrNull())
println(vulnCveGavToClasses.map { it.value.size }.average())
println(vulnCveGavToClasses.map { it.value.size }.median())

6787
1
51
2.038455871519081
1


In [6]:
val depGraphCacheDir = dataDir.resolve("interim/depGraphCache")

val gavToDepGraphCache = vulnGavToClasses
    .map {
        val cachePath = depGraphCacheDir.resolve("${it.key}.vertices.tsv.zip")
        if (!cachePath.isFile()) throw Exception("cache file not found")
        
        it.key to cachePath
    }
    .toMap()
gavToDepGraphCache.count()

3621

In [7]:
import common.DefaultGraph
import io.github.classgraph.ClassInfoList
import java.io.File
import java.net.URLClassLoader
import org.jgrapht.Graph
import org.jgrapht.graph.DefaultEdge
import org.jgrapht.graph.builder.GraphTypeBuilder

fun loadDepGraph(gav: String): DefaultGraph = 
    scripts.exportDepGraphs.loadDepGraphFromCache(gav)

fun loadClassListInfo(gav: String): ClassInfoList =
    scripts.exportDepGraphs.loadClassInfoListFromCache(gav)

fun loadVertexInfo(gav: String) =
    scripts.exportDepGraphs.loadVertexInfo(gav)

### how much of the public API is reaching vulns?

#### first we count if it hits "any" vuln class:

In [8]:
import com.google.common.collect.Queues

/**
 * returns the depths of each visited node from the starting node
 */
fun bfsOnDepGraph(graph: DefaultGraph, startNode: String): Map<String, Int> {
    val expanded = mutableSetOf<String>()
    val depthMap = mutableMapOf<String, Int>()
    val queue = Queues.newArrayDeque<String>()
    queue.add(startNode)
    depthMap[startNode] = 0
    while (!queue.isEmpty()) {
        val node = queue.pop()
        expanded.add(node)
        val parentDepth = depthMap[node]!! 
        graph
            .outgoingEdgesOf(node)
            .map { e -> graph.getEdgeTarget(e) }
            .filterNot { expanded.contains(it) }
            .forEach { v ->
                if (depthMap.containsKey(v)) {
                    depthMap[v] = min(depthMap[v]!!, parentDepth + 1)
                } else {
                    depthMap[v] = parentDepth + 1;
                }
                queue.add(v)
            }
    }
    return depthMap
}

In [9]:
import common.DefaultGraph
import io.github.classgraph.ClassGraph
import io.github.classgraph.ClassInfo
import io.github.classgraph.ClassInfoList
import kotlinx.coroutines.async
import org.jgrapht.alg.shortestpath.FloydWarshallShortestPaths
import org.jgrapht.alg.shortestpath.GraphMeasurer
import org.jgrapht.graph.EdgeReversedGraph
import org.jgrapht.traverse.BreadthFirstIterator
import java.lang.reflect.Modifier
import kotlin.Exception

data class VulnAnalysisInfo(
    val classCount: Int,
    val publicClassCount: Int,
    val vulnClassCount: Int,
    val vulnPublicClassCount: Int,
    val averageDistToVuln: Double,
    val minDistToVuln: Int,
    val maxDistToVuln: Int,
    val medianDistToVuln: Int,
    // val publicClassesWithPublicApiCount: Int,
    // val depGraph: DefaultGraph,
) {
    val apiSurfaceRatio = publicClassCount.toDouble() / classCount.toDouble()
    val vulnApiSurfaceRatio = vulnPublicClassCount.toDouble() / publicClassCount.toDouble()
    val vulnRatio = vulnClassCount.toDouble() / classCount.toDouble()
}

System.gc()

@kotlinx.coroutines.ExperimentalCoroutinesApi fun _vulnInfo() =
    Dispatchers.IO.limitedParallelism(32).let { dispatcher ->
        vulnCveGavToClasses.asSequence().windowed(512, 512, true).flatMapIndexed { batch, w ->
            runBlocking {
                System.gc()
                println("processing batch $batch")
                w.map {
                    async(dispatcher) {
                        val gav = it.key.second
                        val vulnClasses = it.value

                        val depGraph = loadDepGraph(gav)
                        
                        val vertexInfo = loadVertexInfo(gav)

                        val publicClasses =
                            depGraph.vertexSet().filter { v -> Modifier.isPublic(vertexInfo[v]!!) }

                        if (publicClasses.count() > depGraph.vertexSet().count()) {
                            error("#public classes should be <= #classes")
                        }

                        if (publicClasses.count() == 0) {
                            return@async Result.failure(Exception("no public class in $gav"))
                        }

                        val vulnClassesInJar = vulnClasses.intersect(depGraph.vertexSet())
                        if (vulnClassesInJar.size != vulnClasses.size) {
                            error("vuln class number mismatch ${gav}\n***\n$vulnClassesInJar\n***\n$vulnClasses")
                        }

                        val distToNearestVuln = mutableMapOf<String, Int>()
                        vulnClassesInJar.map {
                            // BreadthFirstIterator(EdgeReversedGraph(depGraph), it).asSequence().toSet()
                            bfsOnDepGraph(EdgeReversedGraph(depGraph), it)
                        }.forEach { m ->
                            m.forEach { v, d -> 
                                if (distToNearestVuln.containsKey(v)) {
                                    distToNearestVuln[v] = min(distToNearestVuln[v]!!, d)
                                } else {
                                    distToNearestVuln[v] = d
                                }
                            }
                        }
                        
                        val allVulnClasses = distToNearestVuln.keys.toSet()

                        val vulnPublicClasses = publicClasses.intersect(allVulnClasses)

                        val vulnPublicClassRatio =
                            vulnPublicClasses.count().toDouble() / publicClasses.count().toDouble()

                        if (vulnPublicClassRatio > 1) {
                            error("ratio should be <= 1")
                        }
                        
                        val averageDistToVuln = distToNearestVuln.values.average()
                        val minDistToVuln = distToNearestVuln.values.min()
                        val maxDistToVuln = distToNearestVuln.values.max()
                        val medianDistToVuln = distToNearestVuln.values.median()
                        
                        // val classes = loadClassListInfo(gav)
                        // val classToClassInfo = classes.filter { it.name in depGraph.vertexSet() }.map { it.name to it }.toMap()

                        // val publicClassesWithPublicApi =
                        //     publicClasses
                        //         .filter { c -> classToClassInfo[c]!!.let { it.methodInfo.any { it.isPublic } || it.fieldInfo.any { it.isPublic } } }
                        //         .toSet()

                        val res = VulnAnalysisInfo(
                            classCount = depGraph.vertexSet().count(),
                            publicClassCount = publicClasses.count(),
                            vulnClassCount = allVulnClasses.count(),
                            vulnPublicClassCount = vulnPublicClasses.count(),
                            averageDistToVuln = averageDistToVuln,
                            minDistToVuln = minDistToVuln,
                            maxDistToVuln = maxDistToVuln,
                            medianDistToVuln = medianDistToVuln,
                            // publicClassesWithPublicApiCount = publicClassesWithPublicApi.count()
                            // depGraph = depGraph,
                        )

                        return@async Result.success(res)
                    }
                }.toList().awaitAll().toList()
            }.toList().filter { it.isSuccess }.map { it.getOrNull()!! }.toList()
        }.toList()
    }

val vulnInfo = _vulnInfo()
vulnInfo.count()

processing batch 0
processing batch 1
processing batch 2
processing batch 3
processing batch 4
processing batch 5
processing batch 6
processing batch 7
processing batch 8
processing batch 9
processing batch 10
processing batch 11
processing batch 12
processing batch 13


6787

In [10]:
System.gc()

In [11]:
fun Double.format(digits: Int) =
    "%.${digits}f".format(this)

fun Float.format(digits: Int) =
    "%.${digits}f".format(this)

In [12]:
println("classCount median: ${vulnInfo.map { it.classCount }.median()} avg: ${vulnInfo.map { it.classCount }.average().format(2)}")
println("publicClassCount median: ${vulnInfo.map { it.publicClassCount }.median()} avg: ${vulnInfo.map { it.publicClassCount }.average().format(2)}")
// DISPLAY("publicClassesWithPublicApiCount: ${vulnInfo.map { it.publicClassesWithPublicApiCount }.median()}")
println("vulnClassCount median: ${vulnInfo.map { it.vulnClassCount }.median()} avg: ${vulnInfo.map { it.vulnClassCount }.average().format(2)}")
println("vulnPublicClassCount median: ${vulnInfo.map { it.vulnPublicClassCount }.median()} avg: ${vulnInfo.map { it.vulnPublicClassCount }.average().format(2)}")
println("apiSurfaceRatio median: ${vulnInfo.map { it.apiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.map { it.apiSurfaceRatio * 100 }.average().format(2)}")
// DISPLAY("publicClassesWithPublicApiRatio: ${vulnInfo.map { it.publicClassesWithPublicApiCount.toDouble() / it.classCount.toDouble() }.map { it * 100 }.median()}")
println("vulnApiSurfaceRatio median: ${vulnInfo.map { it.vulnApiSurfaceRatio * 100 }.median().format(2)} avg: ${vulnInfo.map { it.vulnApiSurfaceRatio * 100 }.average().format(2)}")

classCount median: 317 avg: 587.63
publicClassCount median: 286 avg: 539.19
vulnClassCount median: 13 avg: 187.10
vulnPublicClassCount median: 13 avg: 176.65
apiSurfaceRatio median: 93.79 avg: 91.64
vulnApiSurfaceRatio median: 9.07 avg: 26.22


In [13]:
val p =
    letsPlot(mapOf(
        "number of classes" to vulnInfo.map { it.classCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of classes

In [14]:
val p =
    letsPlot(mapOf(
        "number of public classes" to vulnInfo.map { it.publicClassCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of public classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of public classes

In [15]:
val p =
    letsPlot(mapOf(
        "number of vulnerable classes" to vulnInfo.map { it.vulnClassCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of vulnerable classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of vulnerable classes

In [16]:
val p =
    letsPlot(mapOf(
        "number of vuln public classes" to vulnInfo.map { it.vulnPublicClassCount.coerceAtMost(2800) })
    ) +
            geomHistogram(alpha = .3, binWidth = 200, center = 100) { x = "number of vuln public classes"; } +
            xlim(limits = Pair(0, 3000)) +
            ggsize(1800, 800)
p

-100 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 2,100 
 
 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 
 
 2,300 
 
 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 
 2,700 
 
 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 
 2,900 
 
 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 3,100 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,500 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 3,500 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 4,500 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 5,500 
 
 
 
 
 
 
 
 count 
 
 
 
 
 number of vuln public classes

In [17]:
val p =
    letsPlot(mapOf(
        "apiSurfaceRatio" to vulnInfo.map { it.apiSurfaceRatio * 100 })
    ) +
            geomHistogram(alpha = .3, binWidth = 5, center = 2.5) { x = "apiSurfaceRatio"; } +
            xlim(limits = Pair(0, 100)) +
            ggsize(1800, 800)
p

-5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 65 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 75 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 85 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 95 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 105 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 count 
 
 
 
 
 apiSurfaceRatio

In [18]:
val p =
    letsPlot(mapOf(
        "vulnApiSurfaceRatio" to vulnInfo.map { it.vulnApiSurfaceRatio * 100 })
    ) +
            geomHistogram(alpha = .3, binWidth = 5, center = 2.5) { x = "vulnApiSurfaceRatio"; } +
            xlim(limits = Pair(0, 100)) +
            ggsize(1800, 800)
p

-5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 55 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 65 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 75 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 85 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 95 
 
 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 105 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 2,800 
 
 
 
 
 
 
 
 count 
 
 
 
 
 vulnApiSurfaceRatio

what is the depth of vulnerability from public classes?

In [19]:
println("medianDistanceToVuln median: ${vulnInfo.map { it.medianDistToVuln }.median()} avg: ${vulnInfo.map { it.medianDistToVuln }.average().format(2)}")
println("averageDistanceToVuln median: ${vulnInfo.map { it.averageDistToVuln }.median()} avg: ${vulnInfo.map { it.averageDistToVuln }.average().format(2)}")
println("minDistanceToVuln median: ${vulnInfo.map { it.minDistToVuln }.median()} avg: ${vulnInfo.map { it.minDistToVuln }.average().format(2)}")
println("maxDistanceToVuln median: ${vulnInfo.map { it.maxDistToVuln }.median()} avg: ${vulnInfo.map { it.maxDistToVuln }.average().format(2)}")

medianDistanceToVuln median: 2 avg: 2.02
averageDistanceToVuln median: 1.5 avg: 2.01
minDistanceToVuln median: 0 avg: 0.00
maxDistanceToVuln median: 3 avg: 3.92


In [20]:
val p =
    letsPlot(mapOf(
        "medianDistanceToVuln" to vulnInfo.map { it.medianDistToVuln })
    ) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) { x = "medianDistanceToVuln"; } +
            xlim(limits = Pair(0, 10)) +
            ggsize(1800, 800)
p

-0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 4.5 
 
 
 
 
 
 
 
 
 5.0 
 
 
 
 
 
 
 
 
 5.5 
 
 
 
 
 
 
 
 
 6.0 
 
 
 
 
 
 
 
 
 6.5 
 
 
 
 
 
 
 
 
 7.0 
 
 
 
 
 
 
 
 
 7.5 
 
 
 
 
 
 
 
 
 8.0 
 
 
 
 
 
 
 
 
 8.5 
 
 
 
 
 
 
 
 
 9.0 
 
 
 
 
 
 
 
 
 9.5 
 
 
 
 
 
 
 
 
 10.0 
 
 
 
 
 
 
 
 
 10.5 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 500 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 700 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 900 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,100 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,300 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,500 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,700 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 1,900 
 
 
 
 
 
 
 
 count 
 
 
 
 
 medianDistanceToVuln

In [21]:
val p =
    letsPlot(mapOf(
        "averageDistanceToVuln" to vulnInfo.map { it.averageDistToVuln })
    ) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) { x = "averageDistanceToVuln"; } +
            xlim(limits = Pair(0, 10)) +
            ggsize(1800, 800)
p

-0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 4.5 
 
 
 
 
 
 
 
 
 5.0 
 
 
 
 
 
 
 
 
 5.5 
 
 
 
 
 
 
 
 
 6.0 
 
 
 
 
 
 
 
 
 6.5 
 
 
 
 
 
 
 
 
 7.0 
 
 
 
 
 
 
 
 
 7.5 
 
 
 
 
 
 
 
 
 8.0 
 
 
 
 
 
 
 
 
 8.5 
 
 
 
 
 
 
 
 
 9.0 
 
 
 
 
 
 
 
 
 9.5 
 
 
 
 
 
 
 
 
 10.0 
 
 
 
 
 
 
 
 
 10.5 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 600 
 
 
 
 
 
 
 800 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 1,200 
 
 
 
 
 
 
 1,400 
 
 
 
 
 
 
 1,600 
 
 
 
 
 
 
 1,800 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 2,200 
 
 
 
 
 
 
 2,400 
 
 
 
 
 
 
 2,600 
 
 
 
 
 
 
 
 count 
 
 
 
 
 averageDistanceToVuln

In [22]:
val p =
    letsPlot(
        mapOf("minDistanceToVuln" to vulnInfo.map { it.maxDistToVuln })
    ) { x = "minDistanceToVuln" } + ggsize(1800, 800) +
            geomHistogram(alpha = .3, binWidth = 1, center = .5) +
            geomDensity(alpha = 0, size = 1, adjust = 2) { y = "..count.." } + 
            xlim(limits = Pair(0, 30))

p

<path d="M78.2274585394901 297.34286656166967 L78.2274585394901 297.34286656166967 L81.28919859974411 286.37948819255615 L84.35093865999814 275.89965337529105 L87.41267872025215 265.93117485980815 L90.47441878050617 256.49859538573145 L93.53615884076018 247.62308736372813 L96.59789890101419 239.3219064049344 L99.65963896126821 231.6101821257542 L102.72137902152222 224.49840359302368 L105.78311908177625 217.99407066601987 L108.84485914203026 212.10141355769258 L111.90659920228427 206.8215208521608 L114.96833926253828 202.15249647103155 L118.0300793227923 198.0896417737698 L121.09181938304633 194.62565871770005 L124.15355944330034 191.75086983531537 L127.21529950355435 189.45345070737642 L130.27703956380836 187.71967062060344 L133.3387796240624 186.53413719291086 L136.4005196843164 185.88004091839093 L139.46225974457042 185.73939582064918 L142.52399980482443 186.0932727036211 L145.58573986507844 186.92202183531276 L148.64747992533245 188.20518449606095 L151.70921998558646 189.92279019572652 L154.7709600458405 192.05398425065164 L157.8327001060945 194.5781555292948 L160.89444016634852 197.47484341058646 L160.89444016634852 197.47484341058646 L163.95618022660256 200.7238534341949 L167.01792028685657 204.30535195884158 L170.07966034711058 208.199940036282 L173.14140040736459 212.3887070764705 L176.2031404676186 216.85326521562502 L179.2648805278726 221.57576559647418 L182.32662058812662 226.53889802941887 L185.38836064838063 231.72587572183886 L188.45010070863466 237.12040693927992 L191.51184076888867 242.70665559860197 L194.57358082914268 248.46919288490068 L197.6353208893967 254.39294203966506 L200.69706094965073 260.4628807040917 L203.75880100990474 266.66485967421755 L206.82054107015875 272.9843037425127 L209.88228113041276 279.40692556479684 L212.94402119066677 285.9184843590511 L216.0057612509208 292.5047329798246 L219.0675013111748 299.15137170552134 L222.12924137142883 305.84401013064064 L225.19098143168284 312.5681383495942 L228.25272149193685 319.309108409 L231.3144615521909 326.05212678266383 L234.37620161244487 332.7822583944632 L237.4379416726989 339.48444248361136 L240.49968173295292 346.1435203765932 L243.56142179320693 352.7442750026544 L246.62316185346094 359.2714817703137 L249.68490191371498 365.70997021201424 L252.746641973969 372.0443973820694 L255.808382034223 378.26043361216114 L258.870122094477 384.3433004898177 L261.931862154731 390.2788380745886 L264.99360221498506 396.0533750846688 L264.99360221498506 396.0533750846688 L268.055342275239 401.6538324945767 L271.1170823354931 407.0678291593982 L274.1788223957471 412.28378782198916 L277.24056245600116 417.2910398291647 L280.30230251625517 422.0799268804533 L283.3640425765092 426.64189815392115 L286.4257826367632 430.9696011983697 L289.48752269701714 435.0569650500948 L292.5492627572712 438.89927412176837 L295.6110028175252 442.4932315208696 L298.6727428777792 445.83701058569704 L301.73448293803324 448.93029357180404 L304.79622299828725 451.7741044458042 L307.85796305854126 454.3715313132423 L310.91970311879527 456.7267229317643 L313.9814431790493 458.8454950189224 L317.0431832393033 460.7351465372387 L320.1049232995573 462.40440850401285 L323.1666633598113 463.86337754935084 L326.2284034200654 465.1234346871516 L329.2901434803194 466.19714996745864 L332.3518835405734 467.0981738777544 L335.4136236008274 467.8411165499905 L338.4753636610814 468.44141601138926 L341.5371037213354 468.9151968824335 L344.59884378158944 469.2791210779575 L347.66058384184345 469.55023219851233 L350.7223239020975 469.745795412581 L353.7840639623515 469.8831347185449 L356.84580402260553 469.9794695394567 L359.9075440828595 470.05168420003133 L362.96928414311355 470.1164230131224 L366.03102420336756 470.18958044701753 L369.0927642636216 470.28637846506274 L372.1545043238756 470.4211764529095 L375.2162443841296 470.6081280649264 L378.2779844443836 470.8577534200422 L381.3397245046377 471.1821215270591 L384.4014645648917 471.5909998839344 L387.46320462514564 472.09280081596035 L390.524944